In [1]:
import pandas as pd
import os
import json
from tqdm import tqdm
from scipy import stats

orig_df = pd.read_csv("../csv/df.csv")
num_total = len(orig_df)

# Specify dtype for columns 1, 2, and 3
dtype_dict = {"parse": str, "orig_dys": str, "dys": str}

df0 = pd.read_csv("../csv/df-english-fisher-annotations-0.csv", dtype=dtype_dict)
df1 = pd.read_csv("../csv/df-english-fisher-annotations-1.csv", dtype=dtype_dict)

df = pd.concat([df0, df1], ignore_index=True)

# if no words were transcribed, replace nan's with empty string
df["transcript"] = df["transcript"].fillna("")

display(df.head())

,Unnamed: 0.1,Unnamed: 0,show_uri,show_name,show_description,publisher,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,language,transcript,transcript_length,parse,orig_dys,dys
0,0,0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,en,"Hello, hello, hello everyone. This is Katie an...",1716,(FRAG (INTJ (UNK hello)) (INTJ (UNK hello)) (I...,hello _ hello _ hello _ everyone _\nthis _ is ...,hello E hello E hello E everyone _ this _ is _...
1,1,2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,en,Welcome to Inside the 18. Today's episode is a...,2017,(ADJP (UNK welcome) (PP (UNK to) (PP (UNK insi...,welcome _ to _ inside _ the _\ntoday _ 's _ ep...,welcome _ to _ inside _ the _ today _ 's _ epi...
2,2,3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,en,"Hey Cheese fans! Before we get started, I want...",1518,(FRAG (INTJ (UNK hey)) (NP (UNK cheese) (UNK f...,hey _ cheese _ fans _\nbefore _ we _ get _ sta...,hey E cheese _ fans _ before _ we _ get _ star...
3,3,4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,en,"Sorry to interrupt the show, but I do have to ...",1707,(S (ADJP (UNK sorry) (S (VP (UNK to) (VP (UNK ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,sorry _ to _ interrupt _ the _ show _ but _ i ...
4,4,5,spotify:show:5ljREb8VLogQLT7AKGwav1,UPSC Podcasts,Podcasts useful for UPSC aspirants! Mainly dis...,UPSC Podcast,https://anchor.fm/s/8afceec/podcast/rss,spotify:episode:0025w0gdgkl11Nzkmg1wnm,Tourism in India : Opportunities and Challenges,.,13.788000,show_5ljREb8VLogQLT7AKGwav1,0025w0gdgkl11Nzkmg1wnm,en,This is All India Radio. In the program Spotli...,1755,(S (NP (UNK this)) (VP (UNK is) (NP (UNK all) ...,this _ is _ all _ india _ radio _\nin _ the _ ...,this _ is _ all _ india _ radio _ in _ the _ p...


In [2]:
import numpy as np
display(len(df[df["parse"].isna()]))

0

In [3]:
# check on progress
num_completed = len(df.dropna(subset=["parse"]))
print(num_completed)
print(num_total)
print(((num_completed/num_total)*100.0), "percent completed so far")

82601
82601
100.0 percent completed so far


In [4]:
# Function to count occurrences of a specified substring in a string
def count_substring_occurrences(s, substring):
    return s.count("("+substring)

# Apply the function to create a new column with dynamic names
def create_new_count_col(substring_to_count, column_to_search):
    new_column_name = f"{column_to_search}_{substring_to_count}_count"
    df[new_column_name] = df[column_to_search].apply(count_substring_occurrences, substring=substring_to_count)
    
for label in ["INTJ", "EDITED", "PRN", "ADJP", "ADVP", "NP", "PP", "S", "SBAR", "SBARQ", "SINV", "SQ", "VP", "WHADVP", "WHNP", "WHPP", "X"]:
    create_new_count_col(substring_to_count=label, column_to_search="parse")
    
pd.set_option('display.max_columns', None)
display(df)

display(df.describe())

df.to_csv("../csv/df-parse.csv", header=True)

,Unnamed: 0.1,Unnamed: 0,show_uri,show_name,show_description,publisher,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,language,transcript,transcript_length,parse,orig_dys,dys,parse_INTJ_count,parse_EDITED_count,parse_PRN_count,parse_ADJP_count,parse_ADVP_count,parse_NP_count,parse_PP_count,parse_S_count,parse_SBAR_count,parse_SBARQ_count,parse_SINV_count,parse_SQ_count,parse_VP_count,parse_WHADVP_count,parse_WHNP_count,parse_WHPP_count,parse_X_count
0,0,0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,en,"Hello, hello, hello everyone. This is Katie an...",1716,(FRAG (INTJ (UNK hello)) (INTJ (UNK hello)) (I...,hello _ hello _ hello _ everyone _\nthis _ is ...,hello E hello E hello E everyone _ this _ is _...,19,12,8,29,104,609,136,405,109,5,1,4,435,15,37,0,0
1,1,2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,en,Welcome to Inside the 18. Today's episode is a...,2017,(ADJP (UNK welcome) (PP (UNK to) (PP (UNK insi...,welcome _ to _ inside _ the _\ntoday _ 's _ ep...,welcome _ to _ inside _ the _ today _ 's _ epi...,34,16,18,48,106,734,206,467,115,11,2,12,483,15,32,0,8
2,2,3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,en,"Hey Cheese fans! Before we get started, I want...",1518,(FRAG (INTJ (UNK hey)) (NP (UNK cheese) (UNK f...,hey _ cheese _ fans _\nbefore _ we _ get _ sta...,hey E cheese _ fans _ before _ we _ get _ star...,33,23,16,31,90,532,127,365,82,7,0,7,389,15,17,0,1
3,3,4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,en,"Sorry to interrupt the show, but I do have to ...",1707,(S (ADJP (UNK sorry) (S (VP (UNK to) (VP (UNK ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,sorry _ to _ interrupt _ the _ show _ but _ i ...,66,20,7,53,105,626,145,383,86,5,0,6,426,13,19,0,3
4,4,5,spotify:show:5ljREb8VLogQLT7AKGwav1,UPSC Podcasts,Podcasts useful for UPSC aspirants! Mainly dis...,UPSC Podcast,https://anchor.fm/s/8afceec/podcast/rss,spotify:episode:0025w0gdgkl11Nzkmg1wnm,Tourism in India : Opportunities and Challenges,.,13.788000,show_5ljREb8VLogQLT7AKGwav1,0025w0gdgkl11Nzkmg1wnm,en,This is All India Radio. In the program Spotli...,1755,(S (NP (UNK this)) (VP (UNK is) (NP (UNK all) ...,this _ is _ all _ india _ radio _\nin _ the _ ...,this _ is _ all _ india _ radio _ in _ the _ p...,9,10,1,29,56,647,189,289,74,3,0,3,325,10,24,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82596,82596,105355,spotify:show:416U8ZhubKrFHq8ynOaxfH,The Top 10,"Each week, John Rocha and Matt Knost breakdown...",The Top 10,http://thetop10.podomatic.com/rss2.xml,spotify:episode:7zzQnjBXqDApvnm1hLPzVY,The Top 10 - Re-List - Steve Martin Moves,Thanks to our patreon m

,Unnamed: 0.1,Unnamed: 0,duration,transcript_length,parse_INTJ_count,parse_EDITED_count,parse_PRN_count,parse_ADJP_count,parse_ADVP_count,parse_NP_count,parse_PP_count,parse_S_count,parse_SBAR_count,parse_SBARQ_count,parse_SINV_count,parse_SQ_count,parse_VP_count,parse_WHADVP_count,parse_WHNP_count,parse_WHPP_count,parse_X_count
count,82601.00000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000,82601.000000
mean,41300.00000,52733.532996,40.437903,1682.654641,50.897870,16.710100,10.840462,38.783477,87.326860,596.184489,143.622777,388.333592,92.281219,5.505527,0.365359,9.788417,425.756008,14.513904,27.174998,0.136342,0.345311
std,23844.99913,30406.734353,20.097331,267.513855,39.875698,9.584463,9.793306,12.398499,27.166828,98.502935,31.326437,85.953114,25.818001,4.499144,0.753722,7.575123,89.265225,6.674158,10.724439,0.473211,1.323469
min,0.00000,0.000000,10.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20650.00000,26444.000000,23.721117,1530.000000,19.000000,10.000000,4.000000,30.000000,70.000000,540.000000,124.000000,338.000000,76.000000,2.000000,0.000000,4.000000,372.000000,10.000000,20.000000,0.000000,0.000000
50%,41300.00000,52777.000000,38.614783,1702.000000,42.000000,15.000000,9.000000,38.000000,87.000000,600.000000,143.000000,393.000000,92.000000,5.000000,0.000000,8.000000,430.000000,14.000000,26.000000,0.000000,0.000000
75%,61950.00000,79085.000000,54.848433,1859.000000,74.000000,22.000000,15.000000,46.000000,104.000000,659.000000,163.000000,446.000000,109.000000,8.000000,1.000000,14.000000,485.000000,18.000000,33.000000,0.000000,0.000000
max,82600.00000,105359.000000,304.953900,2650.000000,310.000000,119.000000,169.000000,155.000000,432.000000,1278.000000,318.000000,743.000000,221.000000,63.000000,25.000000,79.000000,816.000000,103.000000,245.000000,10.000000,110.000000
